Usage of the package
========

First, some imports

In [1]:
import gzip
import pickle
import random
from collections import namedtuple

from territories import Territory, MissingTreeCache

## Creation of the tree

The first step is to create a tree of known entities. This can be a very compute intensive task, depending on the tree size. That is why, by default, once created, the tree is stored on disk.

Here we will create a very simple tree out of the **tree.txt** file.

In [2]:
Node = namedtuple('Node', ('id', 'parent_id', 'label', 'level'))
split = lambda x: (arg if arg != 'null' else None for arg in x[:-1].split('; '))

try:
    Territory.load_tree()
except MissingTreeCache:
    with open("tree.txt", "r") as file:
        lines = file.readlines()
        stream = ([Node(*split(x) )for x in lines])
        Territory.build_tree(data_stream=stream, save_tree=False)

Then, you can start to create territories from arbitrary territoiral units.

Entities associated territories are represented in an efficient way : if all leaves of a parent node are included in the territory, they are simply replaced by their parent node.

In [3]:
# some node of the tree
print('\n'.join([f"{e.name} | {e.partition_type}" for e in random.sample(Territory.tree.nodes(), 12)]))

Eyne | COM
Saint-Riquier-ès-Plains | COM
Sadeillan | COM
Le Bessat | COM
Drémil-Lafage | COM
Laissey | COM
Moulis | COM
Saint-Pierre-Lafeuille | COM
Secondigny | COM
Champeaux-sur-Sarthe | COM
Le Drennec | COM
Aslonnes | COM


In [4]:
with open("tree_large.gzip", "rb") as file:
    lines = pickle.loads(gzip.decompress(file.read()))

stream = ([Node(*split(x) )for x in lines])
Territory.build_tree(data_stream=stream)

a = Territory.from_names("COM:69123", "COM:93055", "COM:94052")
b = Territory.from_names("COM:27429", "REG:84", "DEP:75")
c = Territory.from_names("COM:38185", "COM:31555", "REG:11")
d = Territory.from_names("COM:33063", "COM:13055", "REG:28")
e = Territory.from_names("COM:35238", "COM:35047", "DEP:27")
f = Territory.from_names("COM:59350", "COM:38442", "REG:53")

You can crate a territory with names of territorial units.

In [5]:
ter = Territory.from_names("DEP:69", "COM:59350", "ARR:75106")
ter

Paris 6e|Lille|Rhône

If names are invalid, an `NotOnTree` exception will be raised.

In [6]:
try:
    Territory.from_names("DEP:69", "do not exist", "garbage")
except Exception as e:
    print(e)

garbage, do not exist where not found in the territorial tree


Territories are jsons serializable, you can simply return them from an API endpoint

In [7]:
import json

print(json.dumps(ter, indent=4))

[
    {
        "name": "Paris 6e",
        "tu_id": "ARR:75106",
        "atomic": true,
        "partition_type": "ARR",
        "postal_code": null
    },
    {
        "name": "Rh\u00f4ne",
        "tu_id": "DEP:69",
        "atomic": false,
        "partition_type": "DEP",
        "postal_code": null
    },
    {
        "name": "Lille",
        "tu_id": "COM:59350",
        "atomic": true,
        "partition_type": "COM",
        "postal_code": null
    }
]


## Operations on territories


Usual operation on territories works as expected :

In [8]:
# addition

print(a, c)
print(a + c)

Lyon|Nogent-sur-Marne|Pantin Grenoble|Toulouse|Île-de-France
Grenoble|Lyon|Toulouse|Île-de-France


In [9]:
# substraction

print(a, d)
print(a - d)

Lyon|Nogent-sur-Marne|Pantin Bordeaux|Marseille|Normandie
Lyon|Nogent-sur-Marne|Pantin


More importantly, sets operations are also supported :

In [10]:
# intersection
print(f"Intersection of {a} and {d} is {a & d}")

# union
print(f"Union of {c} and {f} is {f | c}")

Intersection of Lyon|Nogent-sur-Marne|Pantin and Bordeaux|Marseille|Normandie is {}
Union of Grenoble|Toulouse|Île-de-France and Lille|Saint-Pierre-de-Chartreuse|Bretagne is Grenoble|Lille|Saint-Pierre-de-Chartreuse|Toulouse|Bretagne|Île-de-France


Territorial units may have parents or children, but Territory do not. As a territory may be formed of several territorial units, it has a LCA, a Lowest Common Ancestor.

In [11]:
lyon_and_grenoble = Territory.from_names("COM:38185", "COM:69123")

lyon_and_grenoble.lowest_common_ancestor()

Auvergne-Rhône-Alpes

You can easily retrieve all ancestors of a territory with the `.ancestors()` method, and respectively all of its descendants with the `.descendants()` method :

In [12]:
a.ancestors()

{France,
 Île-de-France,
 Auvergne-Rhône-Alpes,
 Val-de-Marne,
 Seine-Saint-Denis,
 Rhône}

In [13]:
print(a)
a.descendants(include_itself=True)

Lyon|Nogent-sur-Marne|Pantin


{Pantin,
 Nogent-sur-Marne,
 Lyon,
 Lyon 9e,
 Lyon 8e,
 Lyon 7e,
 Lyon 6e,
 Lyon 5e,
 Lyon 4e,
 Lyon 3e,
 Lyon 2e,
 Lyon 1er}

Territories are `True` if they are not empty, but you should probably use the `is_empty()` method for clarity.

In [14]:
if Territory.from_names("DEP:69"):
    print("not empty")

if Territory.from_names():
    print("empty")

if Territory.from_names().is_empty():
    print("empty")

not empty
empty


In [15]:
from itertools import chain

with open("../test.pickle", "rb") as file:
    epci_to_comm = pickle.load(file)


Territory.from_names(*tuple(chain(*epci_to_comm.values())))

FileNotFoundError: [Errno 2] No such file or directory: '../test.pickle'

In [ ]:
min(ter).partition_type

<Partition.DEP: 2>

In [ ]:
ter.type

<Partition.DEP: 2>

In [ ]:
from territories import Partition

s = random.sample(Territory.tree.nodes(), 1000)
ter = Territory.from_names(*(ter.tu_id for ter in s))

len([tu.tu_id for tu in ter.descendants(include_itself=True) if tu.partition_type == Partition.COM])

1155

In [19]:
rhone = Territory.from_name("DEP:69")

Territory.tree.subgraph((rhone.tree_id, )).nodes()

[Rhône]

In [11]:
s = {1, 2, {1, 3, 0}}
s.discard({1, 2})
s

TypeError: unhashable type: 'set'

In [2]:
from territories import Territory

Territory.load_tree()

# ['COM:69083',
#  'COM:69043',
#  'COM:69170',
#  'COM:69190',
#  'COM:69263',
#  'COM:69292',
#  'COM:69034',
 
#  'COM:69161',
#  'COM:69289',

ter = Territory.from_names("DEP:69", 'COM:69191',  'COM:69170')
ss = Territory.tree.successor_indices(next(iter(ter.territorial_units)).tree_id)


called for node France
called for node Auvergne-Rhône-Alpes
